
### Statistical Classification
We want to classify women working and predict wheter or not a new sample is working. This inference is based the set of features shown below. 

* $x_1$: Husband's income
* $x_2$: Years of education
* $x_3$: Years of work experience
* $x_4$: Age
* $x_5$: Number of children < 6 years
* $x_6$: Number of children > 6 years

The dataset will be split (70/30) into training and testing data. Two classification models will be used: logistic regression (parametric model) and k-nearest-neighbors (non-parametric model).

In [2]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

# Plotting configuration
%config InlineBackend.figure_format = 'retina'
plt.style.use(['science', 'notebook', 'grid'])

In [17]:
df = pd.read_csv('data/WomenAtWork.dat', delimiter='\t', engine='python')

split_index = int(np.ceil(0.7 * len(df)))
df_training = df[0:split_index]
df_test = df[split_index:]
df_training.head()

,Work,Constant,HusbandInc,EducYears,ExpYears,Age,NSmallChild,NBigChild
0,1.0,1.0,22.394940,12.0,7.0,43.0,0.0,3.0
1,0.0,1.0,7.232000,8.0,10.0,34.0,0.0,7.0
2,1.0,1.0,18.271990,12.0,4.0,41.0,1.0,5.0
3,0.0,1.0,28.069000,14.0,2.0,43.0,0.0,2.0
4,1.0,1.0,7.799889,12.0,10.0,31.0,0.0,1.0


### Logistic Regression

We begin by applying logistic regression. Logistic regression is a transformed version of linear regression, which limits the output to $[0,1]$, which represents a probability of being in a class or not. Note, we are only making inference on one class.

For logistic regression, we transform the $y_i$ using the Sigmoid function defined as: $f(x) \triangleq \frac{1}{1 + e^x}$. Thus we obtain the following model for our classification problem:
$$
\mathbb{P}({y_i}=1| \mathbf{x}, \mathbf{\theta}) = \frac{1}{1 + \exp \{\theta_0 + \theta_1 x_1 + \ldots + \theta_6 x_6\}} = \frac{1}{1 + \exp \{\mathbf{x}^{\text{T}}\mathbf{\theta}\}}
$$

The sample of observations $\{y_i\}_{i=1}^n$ can now be intepreted as a sequence of Bernoulli random variables. Thus we can obtain the following log-likelihood function rather easily:

$$
-\ell(\bold{X}, \bold{y} | \mathbf{\theta}) = \sum_{i=1}^n \big[(y_i-1)\mathbf{\theta}^{\text{T}}\mathbf{x}_i - \ln(1+\exp\{\mathbf{\theta}^{\text{T}}\mathbf{x}_i\})\big]
$$

Next, we fit the model to the training data using the log-likelihood function above. 

In [58]:
# Create log-likelihood function
def log_likelihood(params: np.array, y: np.array, X: np.array) -> float:
    assert(len(y) == len(X))
    n_observations = len(y)

    log_likelihood = 0
    for i in range(0, n_observations):
        log_likelihood = (log_likelihood 
                          + (y[i] - 1) * params.T @ X[i] 
                          - np.log(1 + np.exp(params.T @ X[i]))
                          )

    return -log_likelihood

In [138]:
from scipy.optimize import minimize

# Format training data in numpy.
y = np.array(df_training[df_training.columns[0]])
X = np.array(df_training[df_training.columns[1:]])
params = 0 * np.ones(7)

# Find MLE parameters.
sol = minimize(log_likelihood, x0=params, args=(y, X))
sol.x
print(sol)

      fun: -5868.388311078871
 hess_inv: array([[ 9.99832205e-01, -5.88539765e-03, -4.12636800e-02,
        -1.20090647e-02, -7.72931440e-04,  2.23093126e-02,
        -1.46074370e-02],
       [-5.88539765e-03,  3.12365277e-02, -1.78575774e-02,
         4.31540619e-02, -3.11629478e-02, -5.69531144e-02,
         3.29055267e-02],
       [-4.12636800e-02, -1.78575774e-02,  4.37715170e-01,
         1.05496357e-01, -1.62324274e-01, -1.59406947e-01,
        -3.12551115e-02],
       [-1.20090647e-02,  4.31540619e-02,  1.05496357e-01,
         2.47152703e-01, -1.67960960e-01, -2.51264373e-01,
         1.65930894e-01],
       [-7.72931440e-04, -3.11629478e-02, -1.62324274e-01,
        -1.67960960e-01,  1.42627796e-01,  1.98858569e-01,
        -1.03236398e-01],
       [ 2.23093126e-02, -5.69531144e-02, -1.59406947e-01,
        -2.51264373e-01,  1.98858569e-01,  1.33976224e+00,
        -1.43210003e-01],
       [-1.46074370e-02,  3.29055267e-02, -3.12551115e-02,
         1.65930894e-01, -1.03236398

/var/folders/d7/rpl_88f12ln2z1s2h4m9ybjm0000gn/T/ipykernel_54986/152974731.py:10: RuntimeWarning: overflow encountered in exp
  - np.log(1 + np.exp(params.T @ X[i]))
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/var/folders/d7/rpl_88f12ln2z1s2h4m9ybjm0000gn/T/ipykernel_54986/152974731.py:10: RuntimeWarning: overflow encountered in exp
  - np.log(1 + np.exp(params.T @ X[i]))


In [161]:
def classifier(data: np.array, params: np.array, threshold: float = 0.5) -> int:
    prob = 1 / (1 + np.exp(params.T @ data))
    if prob > threshold:
        return 1
    else:
        return 0

def classify_data(data: np.array, params: np.array) -> np.array:
    n_observations = len(data)
    prediction = np.zeros(n_observations)

    for i in range(0, n_observations):
        prediction[i] = classifier(data[i], params)

    return prediction

mle_params = sol.x
data = np.array(df_test[df_test.columns[1:]])
validation = np.array(df_test[df_test.columns[0]])
prediction = classify_data(data, mle_params)    

In [162]:
validation

array([1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0.])

In [163]:
prediction

array([0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [164]:
successes = abs(validation-prediction)
successes = np.sum([1 for i in successes if i==0])
accuracy = successes / len(validation)
print(accuracy)

0.24


### Accuracy & Confusion Matrix

### kNN 

### Accuracy & Confusion Matrix

### Comparison